In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
args_batch_size = 64
args_test_batch_size = 1000
args_epochs = 10
args_lr = 0.01
args_momentum = 0.5
args_no_cuda = False
args_seed = 1
args_log_interval = 100

args_cuda = not args_no_cuda and torch.cuda.is_available()

torch.manual_seed(args_seed)
if args_cuda:
    torch.cuda.manual_seed(args_seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if args_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self, p):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(p=p)
        self.p = p
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training, p=self.p)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net(p=0.25)
if args_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args_lr, momentum=args_momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args_log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    train_loss = 0
    train_correct = 0
    test_loss = 0
    test_correct = 0
    for data, target in train_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        train_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    for data, target in test_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        test_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    train_loss /= len(train_loader.dataset)
    test_loss /= len(test_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, test_correct, len(test_loader.dataset),
        100. * test_correct / len(test_loader.dataset)))


for epoch in range(1, args_epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.366173
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.615206
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.805127
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.566315
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.235285
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.488180
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.220668
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.260928
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.236319
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.316104

Train set: Average loss: 0.1642, Accuracy: 57093/60000 (95%)
Test set: Average loss: 0.1544, Accuracy: 9550/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.396924
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.169391
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.138631
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.187013
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.300367
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.347784
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.227688
Train Epoch: 2 [44800/60000 (75%)]	Loss:

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
args_batch_size = 64
args_test_batch_size = 1000
args_epochs = 10
args_lr = 0.01
args_momentum = 0.5
args_no_cuda = False
args_seed = 1
args_log_interval = 100

args_cuda = not args_no_cuda and torch.cuda.is_available()

torch.manual_seed(args_seed)
if args_cuda:
    torch.cuda.manual_seed(args_seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if args_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self, p):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(p=p)
        self.p = p
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training, p=self.p)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net(p=0.5)
if args_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args_lr, momentum=args_momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args_log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    train_loss = 0
    train_correct = 0
    test_loss = 0
    test_correct = 0
    for data, target in train_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        train_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    for data, target in test_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        test_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    train_loss /= len(train_loader.dataset)
    test_loss /= len(test_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, test_correct, len(test_loader.dataset),
        100. * test_correct / len(test_loader.dataset)))


for epoch in range(1, args_epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.376790
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.004478
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.205587
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.931227
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.563367
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.660168
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.491314
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.367745
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.529450
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.554864

Train set: Average loss: 0.2207, Accuracy: 56179/60000 (94%)
Test set: Average loss: 0.2073, Accuracy: 9415/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.570426
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.349697
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.319366
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.324545
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.402807
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.331285
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.405334
Train Epoch: 2 [44800/60000 (75%)]	Loss:

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
args_batch_size = 64
args_test_batch_size = 1000
args_epochs = 10
args_lr = 0.01
args_momentum = 0.5
args_no_cuda = False
args_seed = 1
args_log_interval = 100

args_cuda = not args_no_cuda and torch.cuda.is_available()

torch.manual_seed(args_seed)
if args_cuda:
    torch.cuda.manual_seed(args_seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if args_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self, p):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(p=p)
        self.p = p
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training, p=self.p)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net(p=0.75)
if args_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args_lr, momentum=args_momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args_log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    train_loss = 0
    train_correct = 0
    test_loss = 0
    test_correct = 0
    for data, target in train_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        train_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    for data, target in test_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        test_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    train_loss /= len(train_loader.dataset)
    test_loss /= len(test_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, test_correct, len(test_loader.dataset),
        100. * test_correct / len(test_loader.dataset)))


for epoch in range(1, args_epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.457130
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.252684
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.112433
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.897566
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.612304
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.646706
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.108676
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.059297
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.247025
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.234648

Train set: Average loss: 0.4539, Accuracy: 53497/60000 (89%)
Test set: Average loss: 0.4329, Accuracy: 8986/10000 (90%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.114151
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.988425
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.893794
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.826047
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.135631
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.638794
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.851916
Train Epoch: 2 [44800/60000 (75%)]	Loss:

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
args_batch_size = 64
args_test_batch_size = 1000
args_epochs = 10
args_lr = 0.01
args_momentum = 0.5
args_no_cuda = False
args_seed = 1
args_log_interval = 100

args_cuda = not args_no_cuda and torch.cuda.is_available()

torch.manual_seed(args_seed)
if args_cuda:
    torch.cuda.manual_seed(args_seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if args_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self, p):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(p=p)
        self.p = p
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training, p=self.p)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net(p=1.0)
if args_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args_lr, momentum=args_momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args_log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    train_loss = 0
    train_correct = 0
    test_loss = 0
    test_correct = 0
    for data, target in train_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        train_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    for data, target in test_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        test_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    train_loss /= len(train_loader.dataset)
    test_loss /= len(test_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, test_correct, len(test_loader.dataset),
        100. * test_correct / len(test_loader.dataset)))


for epoch in range(1, args_epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307839
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.306502
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.301817
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.310714
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.304682
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.297040
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.312536
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.296304
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.305298
Train Epoch: 1 [57600/60000 (96%)]	Loss: 2.304569

Train set: Average loss: 2.3270, Accuracy: 6742/60000 (11%)
Test set: Average loss: 2.3282, Accuracy: 1135/10000 (11%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.309081
Train Epoch: 2 [6400/60000 (11%)]	Loss: 2.285652
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.307020
Train Epoch: 2 [19200/60000 (32%)]	Loss: 2.305887
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.290152
Train Epoch: 2 [32000/60000 (53%)]	Loss: 2.295799
Train Epoch: 2 [38400/60000 (64%)]	Loss: 2.307236
Train Epoch: 2 [44800/60000 (75%)]	Loss: 

Train set: Average loss: 0.0364, Accuracy: 59330/60000 (99%)
Test set: Average loss: 0.0382, Accuracy: 9874/10000 (99%)

Train set: Average loss: 0.0555, Accuracy: 59007/60000 (98%)
Test set: Average loss: 0.0539, Accuracy: 9842/10000 (98%)

Train set: Average loss: 0.1075, Accuracy: 58105/60000 (97%)
Test set: Average loss: 0.0964, Accuracy: 9706/10000 (97%)

Train set: Average loss: 2.3272, Accuracy: 6742/60000 (11%)
Test set: Average loss: 2.3283, Accuracy: 1135/10000 (11%)

## Report

|Dropout probability|Train performance|Test performance|
|---|---|---|
|0.25|59330/60000 (99%)|9874/10000 (99%)|
|0.50|59007/60000 (98%)|9842/10000 (98%)|
|0.75|58105/60000 (97%)|9706/10000 (97%)|
|1.00|6742/60000 (11%)|1135/10000 (11%)|

* As the dropout probability increases the performance is decreasing
* In general, if dropout probability is neither too lo nor too high, then it is expected to give better performance, but in this case the performance is decreasing.
* I think it is the case that the dropout proability where high performance is present is below 0.25
* Dropout probability being 1 is effictively removing a layer of network. Hence this performs worse.